# Decision Tree

data

In [1]:
import numpy as np
import scipy.io
import sklearn.metrics
import sklearn 
import os
import random
import pandas as pd
import time
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd
import random
random.seed(2021)
import pickle

In [2]:
path = "../data//train_set"
image_dir = path+"images/"
pt_dir = path+"points/"
label_path = path+"label.csv"

In [3]:
def read_data(path):
  
  # read labels
  labels = pd.read_csv(path+'/label.csv')
  y= labels['label'].to_numpy()

  # read points
  n = 3000
  for i in range(1,n+1):
    p_path = str(i).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords = mat['faceCoordinatesUnwarped'] 
    else:
      cords = mat['faceCoordinates2']

    distance = sklearn.metrics.pairwise_distances(cords)       
          # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords[:,0]), k = 1)]    
          # stretch the upper triangle of the symmetric matrix 
          # to a long array with dimension 3003
          # 3003 = (1+77)*78/2
    if i==1:
      distances = np.mat([flatten_distance])
    else:
      distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
  return (distances, y)

In [4]:
read_time_start=time.time()
X, Y = read_data(path)
print("Read the original dataset takes %s seconds" % round((time.time() - read_time_start),3))
X.shape, Y.shape 

Read the original dataset takes 109.56 seconds


((3000, 3003), (3000,))

In [5]:
print('majority class: %d' % np.sum(Y == 0))
print('minority class: %d' % np.sum(Y == 1))
#imbalanced dataset

majority class: 2402
minority class: 598


# oversample then split data

In [6]:
def data_preprocessing(X, Y, path):

  distances = X
  y = Y

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [7]:
X_balanced, Y_balanced = data_preprocessing(X, Y, path)
X_balanced.shape, Y_balanced.shape

((4804, 3003), (4804,))

In [8]:
print('majority class: %d' % np.sum(Y_balanced == 0))
print('minority class: %d' % np.sum(Y_balanced == 1))
#balanced dataset

majority class: 2402
minority class: 2402


In [10]:
import numpy as np
import matplotlib.pyplot as plt
import smote_variants as sv
import imbalanced_databases as imbd

In [11]:
from sklearn.model_selection import train_test_split
X_balanced_train,X_balanced_test,y_balanced_train,y_balanced_test= train_test_split(X_balanced, Y_balanced,test_size=0.2,random_state=0)
X_balanced_train.shape,X_balanced_test.shape,y_balanced_train.shape,y_balanced_test.shape

((3843, 3003), (961, 3003), (3843,), (961,))

In [12]:
print('majority train class: %d' % np.sum(y_balanced_train == 0))
print('minority train class: %d' % np.sum(y_balanced_train == 1))
print('majority test class: %d' % np.sum(y_balanced_test == 0))
print('minority test class: %d' % np.sum(y_balanced_test == 1))

majority train class: 1918
minority train class: 1925
majority test class: 484
minority test class: 477


In [13]:
# split then oversample

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.20,random_state=0)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
print('majority train class: %d' % np.sum(Y_train == 0))
print('minority train class: %d' % np.sum(Y_train == 1))
print('majority test class: %d' % np.sum(Y_test == 0))
print('minority test class: %d' % np.sum(Y_test == 1))
#imbalanced dataset

majority train class: 1923
minority train class: 477
majority test class: 479
minority test class: 121


In [15]:
def data_preprocessing(X_train, Y_train, path):

  distances = X_train
  y = Y_train

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [16]:
X_train_balanced, y_train_balanced = data_preprocessing(X_train, Y_train, path)
X_train_balanced.shape, y_train_balanced.shape

((3846, 3003), (3846,))

In [17]:
def data_preprocessing(X_test, Y_test, path):

  distances = X_test
  y = Y_test

  n = y.shape[0]
  mat_1 = np.add(np.where(y == 1),1)
  n_oversample = (n-sum(y))-sum(y) 
    # how many samples do we need to generate

  for i in range(n_oversample):
    samples_index = random.sample(list(list(mat_1)[0]), 2)
      # pick two random index of class 1 samples. 

    p_path = str(samples_index[0]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_0 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_0 = mat['faceCoordinates2']
    
    p_path = str(samples_index[1]).zfill(4)+'.mat'
    mat = scipy.io.loadmat(path+'/points/'+p_path)
    if 'faceCoordinatesUnwarped' in mat:
      cords_1 = mat['faceCoordinatesUnwarped'] 
    else:
      cords_1 = mat['faceCoordinates2']

    cords_new = (cords_0 + cords_1) / 2 
        # averaging two sets of cordinates to generate new set of cordinates
    distance = sklearn.metrics.pairwise_distances(cords_new)
        # compute the pairwise distances in each mat
    flatten_distance = distance[np.triu_indices(len(cords_new[:,0]), k = 1)]
        # stretch the upper triangle of the symmetric matrix 
        # to a long array with dimension 3003
        # 3003 = (1+77)*78/2
    
    distances = np.append(distances, np.mat([flatten_distance]), axis = 0)
    y = np.append(y,np.array(1))
        # Append new data to the original dataset

  return (distances, y)

In [18]:
X_test_balanced, y_test_balanced = data_preprocessing(X_test, Y_test, path)
X_test_balanced.shape, y_test_balanced.shape
print('majority train class: %d' % np.sum(y_train_balanced == 0))
print('minority train class: %d' % np.sum(y_train_balanced == 1))
print('majority test class: %d' % np.sum(y_test_balanced == 0))
print('minority test class: %d' % np.sum(y_test_balanced == 1))

majority train class: 1923
minority train class: 1923
majority test class: 479
minority test class: 479


# divide into 2 chunk 3 groups 

        
    • balanced train
    
        1.(train with balanced,test with balanced)
           a.X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
           b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 
        
        2.(train with balanced,test with imbalanced) 
            X_train_balanced.y_train_balanced;X_test,Y_test
      

    • imbalanced train
    
        3. (train with imbalanced,test with imbalanced)
            X_train,Y_train; X_test,Y_test 

model

In [19]:
# 1.a. X_balanced_train,y_balanced_train;X_balanced_test,y_balanced_test
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_balanced_train, y_balanced_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_balanced_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_balanced_test,pre))

Training  model takes 6.16 seconds
Predicting test data takes 0.179 seconds
              precision    recall  f1-score   support

           0       0.65      0.65      0.65       484
           1       0.64      0.65      0.64       477

    accuracy                           0.65       961
   macro avg       0.65      0.65      0.65       961
weighted avg       0.65      0.65      0.65       961



In [21]:
#Save trained NN model
pickle.dump(clf, open("../output/Decision_Tree1a.p",'wb'))


#Load weighted SVM model
pickle.load(open("../output/Decision_Tree1a.p",'rb'))

DecisionTreeClassifier(class_weight='balanced', max_depth=30, max_leaf_nodes=3,
                       min_samples_leaf=2)

In [23]:
# 1. b.X_balanced_train,y_balanced_train;X_test_balanced,y_test_balanced 

from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test_balanced)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(y_test_balanced,pre))

Training  model takes 5.68 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.67      0.32      0.43       479
           1       0.55      0.84      0.67       479

    accuracy                           0.58       958
   macro avg       0.61      0.58      0.55       958
weighted avg       0.61      0.58      0.55       958



In [24]:
#Save trained NN model
pickle.dump(clf, open("../output/Decision_Tree1b.p",'wb'))

#Load weighted SVM model
pickle.load(open("../output/Decision_Tree1b.p",'rb'))

DecisionTreeClassifier(class_weight='balanced', max_depth=30, max_leaf_nodes=3,
                       min_samples_leaf=2)

In [25]:
# 2. X_train_balanced.y_train_balanced;X_test,Y_test
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train_balanced, y_train_balanced)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 5.699 seconds
Predicting test data takes 0.016 seconds
              precision    recall  f1-score   support

           0       0.87      0.32      0.47       479
           1       0.23      0.82      0.36       121

    accuracy                           0.42       600
   macro avg       0.55      0.57      0.42       600
weighted avg       0.74      0.42      0.45       600



In [26]:
#Save trained NN model
pickle.dump(clf, open("../output/Decision_Tree2.p",'wb'))

#Load weighted SVM model
pickle.load(open("../output/Decision_Tree2.p",'rb'))

DecisionTreeClassifier(class_weight='balanced', max_depth=30, max_leaf_nodes=3,
                       min_samples_leaf=2)

In [27]:
#3 X_train,Y_train; X_test,Y_test

from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth = 30, min_samples_leaf=2, max_leaf_nodes=3, class_weight='balanced')
start_time=time.time()
clf = clf.fit(X_train, Y_train)
print("Training  model takes %s seconds" % round((time.time() - start_time),3))

from sklearn.metrics import classification_report
start = time.time()
pre=clf.predict(X_test)
end = time.time()
print("Predicting test data takes %s seconds" % round((end - start),3))
print(classification_report(Y_test,pre))

Training  model takes 3.536 seconds
Predicting test data takes 0.008 seconds
              precision    recall  f1-score   support

           0       0.86      0.71      0.77       479
           1       0.31      0.53      0.39       121

    accuracy                           0.67       600
   macro avg       0.58      0.62      0.58       600
weighted avg       0.75      0.67      0.70       600



In [28]:
#Save trained NN model
pickle.dump(clf, open("../output/Decision_Tree3.p",'wb'))

#Load weighted SVM model
pickle.load(open("../output/Decision_Tree3.p",'rb'))

DecisionTreeClassifier(class_weight='balanced', max_depth=30, max_leaf_nodes=3,
                       min_samples_leaf=2)